#### DNN기반 이진분류 모델 구현
 - 데이터셋 : iris.csv
 - feature : 전부 Sepal_Length, Sepal_Width, Petal_Length, Petal_Width
 - target/label : 1개 Setosa와 나머지
 - 학습방법 : 지도학습 > 분류 > 이진분류
 - 알고리즘 : 인공신경망(ANN) -> MLP, DNN : 은닉층이 많은 구성
 - 프레임워크 : Pytorch

[1] 모듈 로딩 및 데이터 준비
***

In [104]:
# - 모델 관련
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from torchmetrics.classification import F1Score, BinaryF1Score, BinaryConfusionMatrix
from torchinfo import summary

# - 데이터 관련
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import *
from sklearn.model_selection import train_test_split

In [105]:
# 활용 패키지 버전 체크  ==> 사용자 정의 함수로 구현해놓기
def checkversion() :
    print(f'Pytorch v {torch.__version__}')
    print(f'pandas v {pd.__version__}')

In [106]:
#### 데이터 로딩
irisDF = pd.read_csv(r'C:\Users\KDP-17\EX_PANDAS6\TORCH_DL\data\iris.csv')
irisDF['variety'].unique()

array(['Setosa', 'Versicolor', 'Virginica'], dtype=object)

In [107]:
### 타겟 변경 ==> 정수화, 클래스3개 => 2개
irisDF['variety']=(irisDF['variety'] == 'Setosa')
irisDF['variety'] = irisDF['variety'].astype('int')
irisDF['variety'].unique()

array([1, 0])

[2] 모델 클래스 설계 및 정의<hr>
 - 클래스 목적 : iris 데이터 학습 및 추론
 - 클래스 이름 : IrisBCFModel
 - 부모 클래스 : nn.Module
 - 매개변수    : 층별 입/출력 개수 고정 => 매개변수 필요 없음 
 - 속성/필드   :  
 - 클래스 기능 : _ _ init _ _() : 모델 구조 설정, forward() : 순방향 학습  <= 오버라이딩(overriding)
 - 클래스 구조 
   * 입력층 : 입력 4개     출력 10개 (퍼셉트론 10개 존재)
   * 은닉층 : 입력 10개    출력 5개  (퍼셉트론 5개 존재)
   * 출력층 : 입력 5개     출력 1개 (2진분류)

 - 활성화함수
   * 클래스 형태 ==> nn.MELoss, nn.ReLU  -> _ _ init _ _() 메서드에서 사용
   * 함수 형태  ==> torch.nn.functional 아래에 -> forward() 메서드에서 사용


In [108]:
class irisBCFModel(nn.Module) :

    # 모델 구조 구성 및 인스턴스 생성 메서드
    def __init__(self) :
        super().__init__()
        self.in_layer = nn.Linear(4,10)
        self.h_layer = nn.Linear(10,5)
        self.out_layer = nn.Linear(5,1)

    # 순방향 학습 진행 메서드
    def forward(self, data) :
        # 입력층
        y = self.in_layer(data)             # f1w1+f2w2+f3w3+...+f10w10+b
        F.relu(y)                           # relu => y 값의 범위 : 0 < y
        
        # 은닉층 : 10개의 숫자값이 들어옴(>=0)
        y = self.h_layer(y)
        F.relu(y)

        # 출력층 : 30개의 숫자값이 들어옴(>=0)
        return F.sigmoid(self.out_layer(y))

In [109]:
### 모델 인스턴스 생성
model = irisBCFModel()
print(model)

irisBCFModel(
  (in_layer): Linear(in_features=4, out_features=10, bias=True)
  (h_layer): Linear(in_features=10, out_features=5, bias=True)
  (out_layer): Linear(in_features=5, out_features=1, bias=True)
)


In [110]:
### [테스트] 모델 사용 메모리 정보 확인
summary(model, input_size=(10,4))

Layer (type:depth-idx)                   Output Shape              Param #
irisBCFModel                             [10, 1]                   --
├─Linear: 1-1                            [10, 10]                  50
├─Linear: 1-2                            [10, 5]                   55
├─Linear: 1-3                            [10, 1]                   6
Total params: 111
Trainable params: 111
Non-trainable params: 0
Total mult-adds (M): 0.00
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00

[3] 데이터셋 클래스 설계 및 정의 <hr>

In [111]:
class IrisDataset(Dataset) :

    def __init__(self,featureDF, targetDF) :
        self.featureDF = featureDF
        self.targetDF = targetDF
        self.n_rows = featureDF.shape[0]
        self.n_features = featureDF.shape[1]

    def __len__(self) :
        return self.n_rows

    def __getitem__(self, index) :
        
        # 텐서화
        featureTS = torch.FloatTensor(self.featureDF.iloc[index].values)
        targetTS = torch.FloatTensor(self.targetDF.iloc[index].values)

        # 피쳐와 타겟 반환
        return featureTS, targetTS

In [112]:
## [테스트] 데이터셋 인스턴스 생성

# - DataFrame에서 피쳐와 타겟 추출
featureDF = irisDF[irisDF.columns[:-1]]
targetDF = irisDF[irisDF.columns[-1:]]


# - 커스텀 데이터셋 인스턴스 생성
irisDS = IrisDataset(featureDF,targetDF)

# - 데이터로더 인스턴스 생성        <= DataLoader로 테스트 해야함 for문으로 feature랑 label 하나씩 뽑아서 확인하믄 됨
irisDL = DataLoader(irisDS)
for feature, label in irisDL :
    print(feature.shape, label.shape)
    break

torch.Size([1, 4]) torch.Size([1, 1])


[4] 학습 준비 <hr>
 - 학습횟수 : EPOCH          <- 처음 ~ 끝까지 공부하는 단위  
 - 배치크기 : BATCH_SIZE     <- 한번에 학습할 데이터 양
 - 위치지정 : DEVICE         <- 텐서 저장 및 실행 위치 (GPU/CPU)
 - 학습률(lr) : 가중치와 절편 업데이트 시 경사하강법으로 업데이트 간격 설정 (0.001 ~ 0.1) <- 하이퍼파라미터

In [113]:
### 학습 진행 관련 설정값
EPOCH = 1000
BATCH_SIZE = 10
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
LR = 0.001

 - 학습 준비에 필요한 것
    * 인스턴스/객체 : 모델, 데이터셋, 최적화, (, 손실함수, 성능지표)

In [114]:
# 찐 모델 인스턴스 생성
model = irisBCFModel()

# - 학습용, 검증용, 테스트용 데이터셋 분리
X_train, X_test, y_train, y_test = train_test_split(featureDF,targetDF,random_state = 1)
X_train, X_val, y_train, y_val = train_test_split(X_train,y_train,random_state = 1)

# - 학습용, 검증용, 테스트용 인스턴스 생성
trainDS = IrisDataset(X_train,y_train)
valDS = IrisDataset(X_val,y_val)
testDS = IrisDataset(X_test, y_test)

# - 학습용 데이터로더 인스턴스 생성
trainDL = DataLoader(trainDS,batch_size=BATCH_SIZE)

# 최적화 인스턴스 - 최소의 손실을 만드는 w,b 찾는 작업 해주는 인스턴스
# 최적화 인스턴스에 model.parameters() 전달 필요
optimizer = optim.Adam(model.parameters(),lr=LR)


# 손실함수 인스턴스 > 분류 > 이진분류 BinaryCrossEntropyLoss =>BCELoss 
#                                  예측값은 확률값으로 전달 ==> Sigmoid() AF 처리 필요
regLoss = nn.BCELoss()

[5] 학습 진행<hr>

In [115]:
## 학습의 효과 확인을 위해 손실값과 성능평가값 저장 필요
LOSS_HISTORY, SCORE_HISTORY = [[],[]],[[],[]]
CNT = len(trainDL)

for epoch in range(EPOCH) :
    # 학습 모드로 모델 설정
    model.train()


    # 배치크기만큼 데이터 로딩해서 학습 진행
    loss_total , score_total = 0, 0
    for featureTS, targetTS in trainDL :
        
        # 학습 진행
        pre_y = model(featureTS)

        # 손실 계산
        loss = regLoss(pre_y,targetTS)
        loss_total += loss.item()

        # 성능 평가
        score = BinaryF1Score()(pre_y,targetTS,)
        score_total += score.item()

        # 최적화 진행
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    # 에포크 당 검증기능
    # 모델 검증 모드 설정
    model.eval()
    with torch.no_grad():
        # 검증 데이터셋
        val_featureTS = torch.FloatTensor(valDS.featureDF.values)
        val_targetTS = torch.FloatTensor(valDS.targetDF.values)
        # 추론/평가
        pre_val = model(val_featureTS)
        # 손실계산
        loss_val = regLoss(pre_val, val_targetTS)
        # 성능평가
        score_val = BinaryF1Score()(pre_val, val_targetTS)



    # 손실값과 성능평가값 저장
    LOSS_HISTORY[0].append(loss_total/CNT) 
    SCORE_HISTORY[0].append(score_total/CNT)
    LOSS_HISTORY[1].append(loss_val) 
    SCORE_HISTORY[1].append(score_val)  

-  학습 결과 체크 => 학습과 검증의 Loss 변화, 성능 변화 확인